<a href="https://colab.research.google.com/github/vadim-privalov/Neiroset_Novosibirsk/blob/main/%D0%9E%D1%82%D1%81%D0%BB%D0%B5%D0%B6%D0%B8%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_%D0%BC%D1%8F%D1%87%D0%B0_%D1%81_%D0%BF%D0%BE%D0%BC%D0%BE%D1%89%D1%8C%D1%8E_OpenCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Отслеживание мяча с помощью OpenCV

#  Ball Tracking with OpenCV

В этом уроке вы узнаете о том, как отслеживать шары (и другие объекты) в видео с помощью Python и OpenCV. С помощью компьютерного зрения мы будем отрисовывать траекторию движения мяча. 


In this tutorial, you will learn how to track balls (and other objects) in a video using Python and OpenCV. With the help of computer vision, we will draw the trajectory of the ball.

### Скачиваем zip-файл с кодом

### Download the code zip file

In [1]:
!wget http://dataudt.ru/datasets/cv/Lesson_8.Ball_Tracking.zip
!unzip -qq Lesson_8.Ball_Tracking.zip
%cd /content/Lesson_8.Ball_Tracking/ball-tracking

--2022-02-08 09:28:46--  http://dataudt.ru/datasets/cv/Lesson_8.Ball_Tracking.zip
Resolving dataudt.ru (dataudt.ru)... 37.228.117.130
Connecting to dataudt.ru (dataudt.ru)|37.228.117.130|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20309687 (19M) [application/zip]
Saving to: ‘Lesson_8.Ball_Tracking.zip’

Lesson_8.Ball_Track 100%[===================>]  19.37M  4.91MB/s    in 3.9s    

2022-02-08 09:28:51 (4.91 MB/s) - ‘Lesson_8.Ball_Tracking.zip’ saved [20309687/20309687]

/content/Lesson_8.Ball_Tracking/ball-tracking


### Импорт библиотек

### Import Packages

In [2]:
# импортируем необходимые библиотеки
# import the necessary packages
from imutils.video import VideoStream
from collections import deque
import numpy as np
import imutils
import time
import cv2

### Отслеживание мяча с помощью OpenCV


### Ball tracking with OpenCV

In [3]:
# явно укажем аргументы необходимые для работы
# video - путь к видеофайлу 
# output - выходное видео  
# buffer - максимальный размер буфера

# explicitly specify the arguments needed to work
# video - path to the video file 
# output - output video  
# buffer - max buffer size
args = {
	"video": "ball_tracking_example.mp4",
	"output" : "output.avi",
	"buffer": 64
}

In [4]:
# определяем нижнюю и верхнюю границы "зеленого" мяча
# в цветовом пространстве HSV, затем инициализируем список отслеживаемых точек
# define the lower and upper boundaries of the "green"
# ball in the HSV color space, then initialize the
# list of tracked points
greenLower = (29, 86, 6)
greenUpper = (64, 255, 255)
pts = deque(maxlen=args["buffer"])

# берем ссылку на видеофайл и инициализируем указатель на вывод видеофайла
# grab a reference to the video file and initialize pointer to output
# video file
print("[INFO] opening video file...")
vs = cv2.VideoCapture(args["video"])
writer = None

[INFO] opening video file...


In [5]:
# создаем бесконечный цикл
# keep looping
while True:
    # захватываем текущий кадр
	# grab the current frame
	frame = vs.read()[1]

    # если мы просматриваем видео и не захватили кадр, то мы дошли до конца видео
	# if we are viewing a video and we did not grab a frame,
	# then we have reached the end of the video
	if frame is None:
		break

    # изменяем размер кадра, размываем его и преобразовываем в цветовое пространство HSV
	# resize the frame, blur it, and convert it to the HSV
	# color space
	frame = imutils.resize(frame, width=600)
	blurred = cv2.GaussianBlur(frame, (11, 11), 0)
	hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)

    # строим маску для зеленого цвета , затем выполняем
	# серию растяжений и размываний для удаления любых мелких дефектов
	# construct a mask for the color "green", then perform
	# a series of dilations and erosions to remove any small
	# blobs left in the mask
	mask = cv2.inRange(hsv, greenLower, greenUpper)
	mask = cv2.erode(mask, None, iterations=2)
	mask = cv2.dilate(mask, None, iterations=2)

    # находим контуры в маске и инициализируем текущие координаты (x, y) центра мяча
	# find contours in the mask and initialize the current
	# (x, y) center of the ball
	cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,
		cv2.CHAIN_APPROX_SIMPLE)
	cnts = imutils.grab_contours(cnts)
	center = None

    # продолжаем только в том случае, если был найден хотя бы один контур
	# only proceed if at least one contour was found
	if len(cnts) > 0:
        # находим самый большой контур в маске, затем используем его
		# для вычисления минимального охватывающего круга и центра
		# find the largest contour in the mask, then use
		# it to compute the minimum enclosing circle and
		# centroid
		c = max(cnts, key=cv2.contourArea)
		((x, y), radius) = cv2.minEnclosingCircle(c)
		M = cv2.moments(c)
		center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))

        # продолжаем, только если радиус соответствует минимальному размеру
		# only proceed if the radius meets a minimum size
		if radius > 30:
            # рисуем круг и центр на кадре,
			# затем обновляем список отслеживаемых точек
			# draw the circle and centroid on the frame,
			# then update the list of tracked points
			cv2.circle(frame, (int(x), int(y)), int(radius),
				(0, 255, 255), 2)
			cv2.circle(frame, center, 5, (0, 0, 255), -1)
            # обновляем последовательность точек
            # update the points queue
			pts.appendleft(center)


    # перебираем набор отслеживаемых точек
	# loop over the set of tracked points
	for i in range(1, len(pts)):
        # если любая из отслеживаемых точек - None, игнорируем их
		# if either of the tracked points are None, ignore
		# them
		if pts[i - 1] is None or pts[i] is None:
			continue

        # в противном случае вычисляем толщину линии и рисуем соединительные линии
		# otherwise, compute the thickness of the line and
		# draw the connecting lines
		thickness = int(np.sqrt(args["buffer"] / float(i + 1)) * 2.5)
		cv2.line(frame, pts[i - 1], pts[i], (0, 0, 255), thickness)

    # если камеры нет и нам нужно записать выходное видео на диск,
    # то инициализируем камеру
	# if the video writer is None *AND* we are supposed to write
	# the output video to disk initialize the writer
	if writer is None and args["output"] is not None:
		fourcc = cv2.VideoWriter_fourcc(*"MJPG")
		writer = cv2.VideoWriter(args["output"], fourcc, 20,
			(frame.shape[1], frame.shape[0]), True)
  
    # если камера есть, то записываем кадр с распознанными контурами на диск
	# if the writer is not None, write the frame with recognized
	# faces to disk
	if writer is not None:
		writer.write(frame)

# очищаем данные
# do a bit of cleanup
vs.release()

# проверяем, нужно ли очистить кэш камеры
# check to see if the video writer point needs to be released
if writer is not None:
	writer.release()

Наше выходное видео создается в формате .avi. Нам нужно преобразовать его в формат .mp4.

Our output video is created in .avi format. We need to convert it to .mp4 format. 

In [6]:
!ffmpeg -i output.avi output.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [7]:
#@title Воспроизводим выходное видео
#@title Display video inline
from IPython.display import HTML
from base64 import b64encode

mp4 = open("output.mp4", "rb").read()
dataURL = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % dataURL)